In [1]:
import fiona
import pandas as pd
import geopandas as gpd
import csv

In [2]:
from collections import OrderedDict
    
layers = fiona.listlayers('ipbes_pollination_summary_hg_2018-08-16_12_21_-0700_036a5f15f926.gpkg')

OSError: no such file or directory: 'ipbes_pollination_summary_hg_2018-08-16_12_21_-0700_036a5f15f926.gpkg'

In [ ]:
layers

In [ ]:
all_datasets = dict()

for layer in layers:
    all_datasets[layer.lower()] = gpd.read_file(
        "ipbes_pollination_summary_hg_2018-08-16_12_21_-0700_036a5f15f926.gpkg", layer=layer
    )

In [ ]:
all_datasets['relevant_population']

### Fields we want to visualize:
NCP_cur,NCP_sspX, UN_cur, UN_sspX, or any of the changes... cNCP_, cUN_
### Preparing the data

en = energy , fo = Folate , va = vitamineA <br>
curr = current<br>
ssp1 = First scenario<br> <br> 
UN = Unmet need (current, ssp1 , ssp3 , ssp5)<br>
NCP = Nature's Contribution to Pollination(current, ssp1 , ssp3 , ssp5)<br>


In [ ]:
unxncp = all_datasets['unxncp']

current = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_cur','prod_poll_dep_unrealized_fo_1d_cur','prod_poll_dep_unrealized_va_1d_cur',
                  'nat_cont_poll_en_1d_cur','nat_cont_poll_fo_1d_cur','nat_cont_poll_va_1d_cur','prod_poll_dep_realized_en_1d_cur','prod_poll_dep_realized_fo_1d_cur'
                  ,'prod_poll_dep_realized_va_1d_cur','UN_cur','NCP_cur']]

ssp1 = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_ssp1','prod_poll_dep_unrealized_fo_1d_ssp1','prod_poll_dep_unrealized_va_1d_ssp1',
                  'nat_cont_poll_en_1d_ssp1','nat_cont_poll_fo_1d_ssp1','nat_cont_poll_va_1d_ssp1','prod_poll_dep_realized_en_1d_ssp1','prod_poll_dep_realized_fo_1d_ssp1'
                  ,'prod_poll_dep_realized_va_1d_ssp1','UN_ssp1','NCP_ssp1']]

ssp3 = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_ssp3','prod_poll_dep_unrealized_fo_1d_ssp3','prod_poll_dep_unrealized_va_1d_ssp3',
                  'nat_cont_poll_en_1d_ssp3','nat_cont_poll_fo_1d_ssp3','nat_cont_poll_va_1d_ssp3','prod_poll_dep_realized_en_1d_ssp3','prod_poll_dep_realized_fo_1d_ssp3'
                  ,'prod_poll_dep_realized_va_1d_ssp3','UN_ssp3','NCP_ssp3']]

ssp5 = unxncp[['GRIDCODE','prod_poll_dep_unrealized_en_1d_ssp5','prod_poll_dep_unrealized_fo_1d_ssp5','prod_poll_dep_unrealized_va_1d_ssp5',
                  'nat_cont_poll_en_1d_ssp5','nat_cont_poll_fo_1d_ssp5','nat_cont_poll_va_1d_ssp5','prod_poll_dep_realized_en_1d_ssp5','prod_poll_dep_realized_fo_1d_ssp5'
                  ,'prod_poll_dep_realized_va_1d_ssp5','UN_ssp5','NCP_ssp5']]


# Interesting Data 
NCP = unxncp[['GRIDCODE','NCP_cur','NCP_ssp1','NCP_ssp3','NCP_ssp5']]
UN = unxncp[['GRIDCODE','UN_cur','UN_ssp1','UN_ssp3','UN_ssp5']]

# Changes

cNCP = unxncp[['GRIDCODE','cNCP_ssp1','cNCP_ssp3','cNCP_ssp5']]
cUN = unxncp[['GRIDCODE','cUN_ssp1','cUN_ssp3','cUN_ssp5']]



# # Data to csv
# NCP.to_csv('NCP', sep='\t')


In [ ]:
list(all_datasets['pollination_table_full'].columns)

In [ ]:
all_datasets['pollination_table_full']

**Description of the data** <br>

X1 = prod_poll_dep_unrealized (The unrealized production that is pollination dependant)<br>
X2 = prod_poll_dep_realized ( The realized production that is pollination dependant) <br>
Y = prod_poll_indep (production that is pollination independant)<br>
Z = nut_req ( nutrition required for the area)<br>

**Data Available for these timeframes:** 1850 - 1900 - 1910 - 1945 - 1980 - current - ssp1 - ssp3 - ssp5

In [ ]:
pollination_table = all_datasets['pollination_table_full']

#  1850
# Z is not available for this year
data_1850 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1850','prod_poll_dep_unrealized_va_1d_1850','prod_poll_dep_unrealized_fo_1d_1850',
                               'prod_poll_dep_realized_en_1d_1850','prod_poll_dep_realized_va_1d_1850','prod_poll_dep_realized_fo_1d_1850',
                               'prod_total_realized_en_1d_1850','prod_total_realized_va_1d_1850','prod_total_realized_fo_1d_1850']]
data_1850.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1900
# Z is not available for this year
data_1900 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1900','prod_poll_dep_unrealized_va_1d_1900','prod_poll_dep_unrealized_fo_1d_1900',
                               'prod_poll_dep_realized_en_1d_1900','prod_poll_dep_realized_va_1d_1900','prod_poll_dep_realized_fo_1d_1900',
                               'prod_total_realized_en_1d_1900','prod_total_realized_va_1d_1900','prod_total_realized_fo_1d_1900']]
data_1900.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1910
# Z is not available for this year
data_1910 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1910','prod_poll_dep_unrealized_va_1d_1910','prod_poll_dep_unrealized_fo_1d_1910',
                               'prod_poll_dep_realized_en_1d_1910','prod_poll_dep_realized_va_1d_1910','prod_poll_dep_realized_fo_1d_1910',
                               'prod_total_realized_en_1d_1910','prod_total_realized_va_1d_1910','prod_total_realized_fo_1d_1910']]
data_1910.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1945
# Z is not available for this year
data_1945 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1945','prod_poll_dep_unrealized_va_1d_1945','prod_poll_dep_unrealized_fo_1d_1945',
                               'prod_poll_dep_realized_en_1d_1945','prod_poll_dep_realized_va_1d_1945','prod_poll_dep_realized_fo_1d_1945',
                               'prod_total_realized_en_1d_1945','prod_total_realized_va_1d_1945','prod_total_realized_fo_1d_1945']]
data_1945.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']

#  1980
# Z is not available for this year
data_1980 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_1980','prod_poll_dep_unrealized_va_1d_1980','prod_poll_dep_unrealized_fo_1d_1980',
                               'prod_poll_dep_realized_en_1d_1980','prod_poll_dep_realized_va_1d_1980','prod_poll_dep_realized_fo_1d_1980',
                               'prod_total_realized_en_1d_1980','prod_total_realized_va_1d_1980','prod_total_realized_fo_1d_1980']]
data_1980.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo']
data_1980

#  cur
data_cur = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_cur','prod_poll_dep_unrealized_va_1d_cur','prod_poll_dep_unrealized_fo_1d_cur',
                               'prod_poll_dep_realized_en_1d_cur','prod_poll_dep_realized_va_1d_cur','prod_poll_dep_realized_fo_1d_cur',
                               'prod_total_realized_en_1d_cur','prod_total_realized_va_1d_cur','prod_total_realized_fo_1d_cur',
                               'nut_req_en_1d_cur','nut_req_va_1d_cur','nut_req_fo_1d_cur']]

data_cur.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']

#  ssp1
data_ssp1 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_ssp1','prod_poll_dep_unrealized_va_1d_ssp1','prod_poll_dep_unrealized_fo_1d_ssp1',
                               'prod_poll_dep_realized_en_1d_ssp1','prod_poll_dep_realized_va_1d_ssp1','prod_poll_dep_realized_fo_1d_ssp1',
                               'prod_total_realized_en_1d_ssp1','prod_total_realized_va_1d_ssp1','prod_total_realized_fo_1d_ssp1',
                               'nut_req_en_1d_ssp1','nut_req_va_1d_ssp1','nut_req_fo_1d_ssp1']]

data_ssp1.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']

#  ssp3
data_ssp3 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_ssp3','prod_poll_dep_unrealized_va_1d_ssp3','prod_poll_dep_unrealized_fo_1d_ssp3',
                               'prod_poll_dep_realized_en_1d_ssp3','prod_poll_dep_realized_va_1d_ssp3','prod_poll_dep_realized_fo_1d_ssp3',
                               'prod_total_realized_en_1d_ssp3','prod_total_realized_va_1d_ssp3','prod_total_realized_fo_1d_ssp3',
                               'nut_req_en_1d_ssp3','nut_req_va_1d_ssp3','nut_req_fo_1d_ssp3']]

data_ssp3.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']

#  ssp5
data_ssp5 = pollination_table[['GRIDCODE','country',
                               'prod_poll_dep_unrealized_en_1d_ssp5','prod_poll_dep_unrealized_va_1d_ssp5','prod_poll_dep_unrealized_fo_1d_ssp5',
                               'prod_poll_dep_realized_en_1d_ssp5','prod_poll_dep_realized_va_1d_ssp5','prod_poll_dep_realized_fo_1d_ssp5',
                               'prod_total_realized_en_1d_ssp5','prod_total_realized_va_1d_ssp5','prod_total_realized_fo_1d_ssp5',
                               'nut_req_en_1d_ssp5','nut_req_va_1d_ssp5','nut_req_fo_1d_ssp5']]

data_ssp5.columns = ['GRIDCODE','name','X1_en','X1_va','X1_fo','X2_en','X2_va','X2_fo','X2+Y_en','X2+Y_va','X2+Y_fo','Z_en','Z_va','Z_fo']





In [ ]:
def add_iso_columns (dataFrame,country_iso):
    dataFrame = pd.merge(dataFrame, country_iso,left_on='name',right_on='English short name lower case', how='outer').head(dataFrame.shape[0])
    dataFrame= dataFrame.drop(columns=['English short name lower case', 'Alpha-2 code','Numeric code','ISO 3166-2','Unnamed: 0'])
    dataFrame = dataFrame.rename(index=str, columns={"Alpha-3 code": "iso3"})

    cols = dataFrame.columns.tolist()
    cols = [cols[0]] + [cols[-1]] + cols[1:-1]
    dataFrame = dataFrame[cols]
    return dataFrame

def add_iso (dataFrame,country_iso, name):
    dataFrame = dataFrame.groupby(['name']).sum()
    dataFrame.reset_index(level=0, inplace=True)
    dataFrame= add_iso_columns(dataFrame,country_iso)
    return dataFrame
    
def create_csv(dataFrame,country_iso, name):
    dataFrame = add_iso(dataFrame,country_iso, name)
    dataFrame.to_csv(name +'.csv', sep=',')
    
def create_all_csv(data_folder):
    create_csv (data_1850,country_iso, data_folder+'1850')
    create_csv (data_1900,country_iso, data_folder+'1900')
    create_csv (data_1910,country_iso, data_folder+'1910')
    create_csv (data_1945,country_iso, data_folder+'1945')
    create_csv (data_1980,country_iso, data_folder+'1980')
    create_csv (data_cur,country_iso, data_folder+'current')
    create_csv (data_ssp1,country_iso, data_folder+'ssp1')
    create_csv (data_ssp3,country_iso, data_folder+'ssp3')
    create_csv (data_ssp5,country_iso, data_folder+'ssp5')
    


In [ ]:
country_iso = pd.read_csv("country_iso.csv")
data_folder = 'data_iso/'
create_all_csv(data_folder)

In [ ]:
# X2+Y folate for all the periods
mock_data = pd.DataFrame(index=data_1850.index)
mock_data = pd.concat([data_1850.name,data_1850['X2+Y_fo'],
                      data_1900['X2+Y_fo'],data_1910['X2+Y_fo'],
                      data_1945['X2+Y_fo'],data_1980['X2+Y_fo'],
                      data_cur['X2+Y_fo'],data_ssp1['X2+Y_fo'],
                      data_ssp3['X2+Y_fo'],data_ssp5['X2+Y_fo'],], axis=1)
mock_data.columns = ['name', '1850','1900', '1910','1945', '1980','cur', 'ssp1','ssp3', 'ssp5']
mock_data = add_iso (mock_data,country_iso, data_folder+'mock_data')
mock_data.to_csv('mock_data' +'.csv', sep=',')

world_data = pd.DataFrame(mock_data.sum()).transpose().drop(columns=['name','iso3'])
world_data.to_csv('world_data' +'.csv', sep=',')
mock_data

Questions:

1. What does the GRIDCODE actually mean? and if they are the regions how can we relate them to latitude and longitude?
2. SSP1, SSP3, SSP5 are they the future scenario description? but how are these scenarios described?
3. Difference between semi natural and natural habitats?
4. 10s – globio, 30s – gpw, monfreda 2008 – 5m, ssp – 1/8d) clearing scales?
5. What does `c_` prefix in front of columns mean?

Observations:

- GRIDCODE is unique.